In [1]:
from random import choice
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [ ]:
# returns a symbolic object 
def symbolize(real_object):
    name = real_object.__name__   
    sy_obj = symbolic_object()
    sy_obj.set_name(name)
    return sy_obj